<a href="https://colab.research.google.com/github/tansaurus/VCMI/blob/main/Saving_ADC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os
import glob
import zipfile
import random
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset

In [ ]:
batch_size = 32
epochs = 20
lr = 1e-4
gamma = 0.7
seed = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(seed)

In [ ]:
device = 'cuda'

In [ ]:
os.makedirs('data', exist_ok=True)

In [ ]:
base_dir = 'data'
train_g1_dir = os.path.join(base_dir, 'train/g1')
train_g2_dir = os.path.join(base_dir, 'train/g2')
test_g1_dir = os.path.join(base_dir, 'test/g1')
test_g2_dir = os.path.join(base_dir, 'test/g2')
val_g1_dir = os.path.join(base_dir, 'val/g1')
val_g2_dir = os.path.join(base_dir, 'val/g2')

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/dataset/SN_Cropped_ADC.zip') as data_zip:
     data_zip.extractall('data')

In [ ]:
train_g1_list = glob.glob(os.path.join(train_g1_dir, '*.png'))
train_g2_list = glob.glob(os.path.join(train_g2_dir, '*.png'))
test_g1_list = glob.glob(os.path.join(test_g1_dir, '*.png'))
test_g2_list = glob.glob(os.path.join(test_g2_dir, '*.png'))
val_g1_list = glob.glob(os.path.join(val_g1_dir, '*.png'))
val_g2_list = glob.glob(os.path.join(val_g2_dir, '*.png'))

In [ ]:
# 리스트 내용 확인
print(f'Train G1 list: ',len(train_g1_list))
print(f'Train G2 list: ',len(train_g2_list))
print(f'Test G1 list: ',len(test_g1_list))
print(f'Test G2 list: ',len(test_g2_list))
print(f'Val G1 list: ',len(val_g1_list))
print(f'Val G2 list: ',len(val_g2_list))

Train G1 list:  168
Train G2 list:  48
Test G1 list:  160
Test G2 list:  48
Val G1 list:  72
Val G2 list:  11


In [ ]:
import random
from torchvision import transforms
from PIL import Image
import os

# 48개 이미지 중에서 중복을 허용하여 120개 선택
random.seed(seed)  # 일관된 결과를 위한 시드 설정
selected_g2_images = [random.choice(train_g2_list) for _ in range(288)]

augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.2, 0.2)),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_g2_augmented = []

# 선택된 이미지를 각각 1번 증강
for idx, img_path in enumerate(selected_g2_images):
    img = Image.open(img_path)
    seed_everything(seed + idx)  # 각 증강마다 다른 시드 사용
    augmented_img = augmentation_transforms(img)
    augmented_img_pil = transforms.ToPILImage()(augmented_img)
    train_g2_augmented.append((augmented_img_pil, 1))

# train_g2_augmented의 이미지를 저장
train_g2_augmented_dir = train_g2_dir  # 저장 경로 설정

for idx, (image, label) in enumerate(train_g2_augmented):
    image_path = os.path.join(train_g2_augmented_dir, f"augmented_image_{idx}.png")
    image.save(image_path)  # 이미지 저장
    train_g2_augmented[idx] = (image_path, label)  # 저장된 이미지 경로로 변경


In [ ]:
# g1은 레이블 0, g2는 레이블 1로 할당
train_g2_list = train_g2_list + train_g2_augmented

train_g1_data = [(path, 0) for path in train_g1_list]
train_g2_data = [(path, 1) for path in train_g2_list]

test_g1_data = [(path, 0) for path in test_g1_list]
test_g2_data = [(path, 1) for path in test_g2_list]

val_g1_data = [(path, 0) for path in val_g1_list]
val_g2_data = [(path, 1) for path in val_g2_list]


In [ ]:
train_data = train_g1_data + train_g2_data
test_data = test_g1_data + test_g2_data
valid_data = val_g1_data + val_g2_data

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # 이미지를 흑백으로 변환하고 채널 수를 3으로 설정합니다.
    transforms.ToTensor(),
])

class T2wMRDataset(torch.utils.data.Dataset):
    def __init__(self, file_list):
        self.file_list = file_list  # file_list는 [(img_path, label), ...] 형태

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        np.random.seed(seed)
        torch.manual_seed(seed)
        img_path, label = self.file_list[idx]
        return img_path, label  # 경로와 레이블을 튜플로 반환

In [ ]:
train_dataset = T2wMRDataset(train_data)
test_dataset = T2wMRDataset(test_data)
val_dataset = T2wMRDataset(valid_data)

print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

504
208
83


In [ ]:
import csv
import os
import shutil

def save_image_paths_labels(loader, dest_dir, csv_filename):
    # dest_dir이 존재하지 않으면 생성
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir, exist_ok=True)

    # CSV 파일 경로
    csv_path = os.path.join(dest_dir, csv_filename)

    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["image_path", "label"])

        for img_paths, labels in loader:
            for img_path, label in zip(img_paths, labels):
                # img_path에서 실제 경로 문자열을 추출합니다.
                img_path_str = img_path[0] if isinstance(img_path, tuple) else img_path
                file_name = os.path.basename(img_path_str)
                dest_path = os.path.join(dest_dir, file_name)

                # 이미지를 dest_dir로 복사합니다.
                shutil.copy(img_path_str, dest_path)

                # CSV 파일에 경로와 레이블을 기록합니다.
                writer.writerow([dest_path, int(label)])


# DataLoader 인스턴스 생성
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# 각 DataLoader에 대해 실행
save_image_paths_labels(train_loader, '/content/drive/My Drive/Colab Notebooks/VCMI/monomodal_HGG/ADC/train', 'train_labels.csv')
save_image_paths_labels(test_loader, '/content/drive/My Drive/Colab Notebooks/VCMI/monomodal_HGG/ADC/test', 'test_labels.csv')
save_image_paths_labels(val_loader, '/content/drive/My Drive/Colab Notebooks/VCMI/monomodal_HGG/ADC/val', 'val_labels.csv')
